# Flagging

Demonstration of prototype ngCASA flagging functions. This is work in progress.

This walkthrough is designed to be run in a Jupyter notebook on Google Colaboratory. To open the notebook in colab, go [here](https://colab.research.google.com/github/FedeMPouzols/cngi_prototype/blob/ngcasa_flagging_experiments/docs/flagging.ipynb)



## Get required packages and data

In [ ]:
# Installation
import os
# (WIP note: we might get by without casatools, if we prepare some .vis.zarr available for download re-check)
print("installing casa6 + cngi (takes a minute or two)...")
os.system("apt-get install libgfortran3")
os.system("pip install --index-url https://casa-pip.nrao.edu/repository/pypi-casa-release/simple casatools==6.1.0.118")

os.system("pip install cngi-prototype==0.0.67")

In [ ]:
# Retrieve and extract demonstration datasets
print('retrieving MS tarfiles...')
!gdown -q --id 15HfB4rJKqEH7df088Ge5YLrCTXBIax6R
!gdown -q --id 1N9QSs2Hbhi-BrEHx5PA54WigXt8GGgx1
print('extracting MS tarfiles...')
!tar -xf M100.ms.tar
!tar -xzf sis14_twhya_calibrated_flagged.ms.tar.gz
# TODO: download the ".flagonline.txt" file
print('complete')

## Initialize the Environment
Omitting for now. I think we don't need it here and it shouldn't show up here at the ngCASA level anyway.

## Load demo data

In [ ]:
from cngi.conversion import convert_ms

zarr_name = 'twhya.vis.zarr' 
multi_xds = convert_ms('sis14_twhya_calibrated_flagged.ms', outfile=zarr_name) 

In [ ]:
from cngi.dio import describe_vis 
describe_vis(zarr_name)
# We will be using the only SPW present in this dataset
vis_dset = multi_xds.xds0

## Flag summaries
Summaries of flags by different dimensions in a dictionary, similar to the dictionary produce by CASA flagdata in 'summary' mode.

In [ ]:
from ngcasa.flagging import summary
counts = summary(vis_dset)
print(counts)

## Handling flag versions




In [ ]:
from ngcasa.flagging import manager_list, manager_add, manager_remove
versions = manager_list(vis_dset)
print(versions)

vis_flags = manager_add(vis_dset, 'start', 'flags state at start')
# vis_flags = manual_unflag(vis_flags, [{'time': slice('2011-09-16T15:38:17','2011-10-16T18:39:50')}])
vis_flags = manager_add(vis_flags, 'backup', 'backup description')
#
vis_flags = manager_add(vis_flags, 'final', 'backup second descr')
#
versions = manager_list(vis_flags)
print(versions)

# fxds['FLAG'] = fxds['FLAG_final']
fxds['FLAG'] = fxds['FLAG_backup'] | fxds['FLAG_start']

# We can always drop versions that are no longer useful
vis_flags = manager_remove(vis_flags, 'start')
vis_flags = manager_remove(vis_flags, 'backup')
vis_flags = manager_remove(vis_flags, 'final')
versions = manager_list(vis_flags)
print(versions)

## Running flagging methods
A few illustrative examples, trying to cover all the categories of flagging methods.



### Manual flagging and meta-information based methods
Methods based on data selection and/or meta-information. Simple examples with data selection based flagging and unflagging.

In [ ]:
from ngcasa.flagging import manual_flag, manual_unflag

#
vis_dset = manager_add(vis_dset, 'checkpointA', 'after a couple of example manual selections')
#
vis_dset = manager_add(vis_dset, 'manual_sels', 'after applying a few manual flags')

vis_unflagged = manual_unflag(vis_dset, {})
counts = summary(vis_unflagged)
print('After unflagging: {}'.format(counts))
flagged_manual = applyflags(vis_dset, flags=['FLAG'])
visplot(flagged_manual.DATA, plot_axes)

### Manual flagging with command lists
An example of application of a list of manual flagging commands, resembling use cases from pipelines. An additional required input is the file of flagging commands. Here we use a `.flagonline.txt` file as used in pipelines, where we usually find of the order of 1000s or 10s of thousands of commands. The `.flagonline.txt` is the lion's share of the full list of commands used by pipelines (`.flagcmds.txt`), where the `.flagonline.txt` list of commands is extended with a much shorter list of additional commands that may include a number of summaries, selections based on intent and frequency, the shadow method, etc.

The selection syntax used is the Xarray selection syntax (see examples in the [Visibilities walkthrough ](visibilities.ipynb)), with selection by label `xds.sel(...)`.

In [ ]:
vis_flags = manual_unflag(vis_flags, [{'time': slice('2011-09-16T15:38:17','2011-10-16T18:39:50')}])
#
vis_flags = manual_unflag(vis_flags, [{)}])

# Flag two groups of adjacent ~20 chans
vis_flags = manual_flag(vis_flags, [{'chan': slice(113.8e9, 113.95e9)}, 
                                    {'chan': slice(115.0e9, 115.16e9)}]

# Flag some baselines, by ID
vis_flgs = manual_flag(vis_flags, [{'baseline': [133, 134, 135]}])
# Flag polarization, by ID
# vis_flgs = manual_flag(vis_flags, [{'pol': 9}])

# vis_dset = manual_flag(vis_dset, cmdfile='.flagonline.txt')
vis_dset = manager_add(vis_dset, 'manual_list', 'after applying list of selection commands')

### Auto-flagging methods
An illustrative example using the `auto_clip` method. Other auto-flagging methods such as tfcrop, rflag, and uvbin are not implemented.

In [ ]:
from ngcasa.flagging import auto_clip
# 
vis_dset = manager_add(vis_dset, 'auto_clip', 'after applying clip')

## Applying flags
To apply a version of flags on a visibilities dataset, before going on to further processing, the function `cngi.vis.applyflags` should be applied. Some examples can be found in the [Continuum Imaging Example](imaging/continuum_imaging_example.ipynb) or the [Visibilities walkthrough](visibilities.ipynb). `cngi.vis.applyflags` sets the flagged data values to NaN. This has the effect that those NaN values are effectively excluded from subsequent CNGI/ngCASA processing. Other components of CNGI and ngCASA, such as imaging, will ignore those NaN values.

In [ ]:
from cngi.vis import applyflags
from cngi.vis import visplot

versions = manager_list(vis_dset)
versions

plot_coords = ['time', 'chan']
visplot(vis_dset.DATA, plot_axes)
vis_dataset_flagged = applyflags(vis_dataset, flags=['FLAG'])
# Further processing: visualization, calibration, imaging, etc. with flags applied (flagged data excluded)
# ...
# TODO: a plot example like in the visibilities notebook

flagged_manual = applyflags(vis_dset, flags=['FLAG_MANUAL_LIST'])
visplot(flagged_manual.DATA, plot_axes)

flagged_clip = applyflags(vis_dset, flags=['FLAG_AUTO_CLIP'])
visplot(flagged_clip.DATA, plot_axes)